# Curacao Risk Level for Extreme Sea Level Hazard Using Gornitz et al. (1994)
# Curacao Multi-Criteria Risk Level for Extreme Sea Level Hazard Using Mclaughlin and Cooper (2010)


In [23]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from scipy.spatial import KDTree
import numpy as np

# Additional imports
import pystac_client
import xarray as xr
import pathlib
import os
import geopandas as gpd
import matplotlib as mpl
from matplotlib import colors
import rasterio

import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
from rasterio.plot import show

import matplotlib.colors as mcolors
from shapely.geometry import Point
import pandas as pd

from tqdm import tqdm

import webbrowser
from pathlib import Path
import os
import time

from itertools import compress
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from tqdm import tqdm

from shapely.geometry import box



In [24]:
Exposure_df_kmeans=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Exposure\Caribbean_Exposure_classified_kmeans.csv")
# Exposure_df["Sediment_Type - Exposure level"]=Exposure_df["Sediment Type"]


Exposure_df_kmeans

,transect_id,country_id,continent,country_name,Percentage of Built Up Area_kmeans,LECZ Area÷LECZ mean elevation_kmeans,Neashore Slope (tanβ or m^-1)_kmeans,Population Within 1km of the coastline_kmeans,Sediment Type_kmeans,geometry,Percentage of Built Up Area_kmeans - Exposure level_Kmeans,LECZ Area÷LECZ mean elevation_kmeans - Exposure level_Kmeans,Neashore Slope (tanβ or m^-1)_kmeans - Exposure level_Kmeans,Population Within 1km of the coastline_kmeans - Exposure level_Kmeans,Sediment Type_kmeans - Exposure level_Kmeans
0,BOX_116_353_2,COL,South America,Colombia,0.0,1.366501e+06,11.291917,0.0,4.0,POINT (-71.8798045985 12.250683718),1,2,4.0,1,4.0
1,BOX_116_353_3,COL,South America,Colombia,0.0,1.974146e+06,13.060951,0.0,2.0,POINT (-71.87698143850001 12.247308305),1,3,4.0,1,2.0
2,BOX_116_353_4,COL,South America,Colombia,0.0,3.834474e+06,13.060951,0.0,4.0,POINT (-71.8756318462 12.2431443263),1,4,4.0,1,4.0
3,BOX_116_353_5,COL,South America,Colombia,0.0,5.239073e+06,11.420912,0.0,2.0,POINT (-71.8752334021 12.2387721729),1,4,4.0,1,2.0
4,BOX_116_353_6,COL,South America,Colombia,0.0,6.112962e+06,11.420912,0.0,2.0,POINT (-71.8748349513 12.2343999454),1,5,4.0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,DOM,North America,Dominican Republic,0.0,1.793014e+06,3.867398,1593.0,4.0,POINT (-71.6681862112 17.9652589019),1,2,3.0,2,4.0
37505,BOX_139_002_62,DOM,North America,Dominican Republic,0.0,1.861771e+06,3.867398,2227.0,4.0,POINT (-71.6662284163 17.9614175026),1,2,3.0,2,4.0
37506,BOX_139_002_63,DOM,North America,Dominican Republic,0.0,1.970466e+06,3.790441,2710.0,4.0,POINT (-71.664582619 17.9574419454),1,3,3.0,2,4.0
37507,BOX_139_002_64,DOM,North America,Dominican Republic,0.0,2.126400e+06,3.790441,2617.0,4.0,POINT (-71.6629367897 17.953466310899998),1,3,3.0,2,4.0


# Add the kmeans for built up percentage, Nearshore slope and population

In [25]:
Exposure_df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Exposure\Caribbean_Exposure_classified_percentile.csv")
# Exposure_df["Sediment_Type - Exposure level"]=Exposure_df["Sediment Type"]




Exposure_df.iloc[:,10]=Exposure_df_kmeans.iloc[:,10] #built up %
Exposure_df.iloc[:,13]=Exposure_df_kmeans.iloc[:,13] #pop
Exposure_df.iloc[:,12]=Exposure_df_kmeans.iloc[:,12] #ns
Exposure_df

,transect_id,country_id,continent,country_name,Percentage of Built Up Area_percentile,LECZ Area÷LECZ mean elevation_percentile,Neashore Slope (tanβ or m^-1)_percentile,Population Within 1km of the coastline_percentile,Sediment Type_percentile,geometry,Percentage of Built Up Area_percentile - Exposure level_percentile,LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile,Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile,Population Within 1km of the coastline_percentile - Exposure level_percentile,Sediment Type_percentile - Exposure level_percentile
0,BOX_116_353_2,COL,South America,Colombia,0.0,1.366501e+06,11.291917,0.0,4.0,POINT (-71.8798045985 12.250683718),1,4,4.0,1,4.0
1,BOX_116_353_3,COL,South America,Colombia,0.0,1.974146e+06,13.060951,0.0,2.0,POINT (-71.87698143850001 12.247308305),1,5,4.0,1,2.0
2,BOX_116_353_4,COL,South America,Colombia,0.0,3.834474e+06,13.060951,0.0,4.0,POINT (-71.8756318462 12.2431443263),1,5,4.0,1,4.0
3,BOX_116_353_5,COL,South America,Colombia,0.0,5.239073e+06,11.420912,0.0,2.0,POINT (-71.8752334021 12.2387721729),1,5,4.0,1,2.0
4,BOX_116_353_6,COL,South America,Colombia,0.0,6.112962e+06,11.420912,0.0,2.0,POINT (-71.8748349513 12.2343999454),1,5,4.0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,DOM,North America,Dominican Republic,0.0,1.793014e+06,3.867398,1593.0,4.0,POINT (-71.6681862112 17.9652589019),1,4,3.0,2,4.0
37505,BOX_139_002_62,DOM,North America,Dominican Republic,0.0,1.861771e+06,3.867398,2227.0,4.0,POINT (-71.6662284163 17.9614175026),1,4,3.0,2,4.0
37506,BOX_139_002_63,DOM,North America,Dominican Republic,0.0,1.970466e+06,3.790441,2710.0,4.0,POINT (-71.664582619 17.9574419454),1,5,3.0,2,4.0
37507,BOX_139_002_64,DOM,North America,Dominican Republic,0.0,2.126400e+06,3.790441,2617.0,4.0,POINT (-71.6629367897 17.953466310899998),1,5,3.0,2,4.0


In [26]:
Vulnerability_df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Vulnerability\Caribbean_Vulnerability_classified_percentile.csv")

Vulnerability_df

,transect_id,country_id,continent,country_name,Infant Mortality Rate (3_yr_Average)_Percentile,GDP per Capita (3_yr_Average)_Percentile,Mean Years of Schooling_Percentile,Deaths from Recent Shocks (2004-2023)_Percentile,World Protected Areas (Status and Governance)_Percentile,geometry,Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile,GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile,Mean Years of Schooling_Percentile - Vulnerability level_percentile,Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile,World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile
0,BOX_116_353_2,COL,South America,Colombia,11.433333,5974.501814,8.862860,3143,3,POINT (-71.8798045985 12.250683718),3.0,4,4.0,5,3
1,BOX_116_353_3,COL,South America,Colombia,11.433333,5974.501814,8.862860,3143,3,POINT (-71.87698143850001 12.247308305),3.0,4,4.0,5,3
2,BOX_116_353_4,COL,South America,Colombia,11.433333,5974.501814,8.862860,3143,3,POINT (-71.8756318462 12.2431443263),3.0,4,4.0,5,3
3,BOX_116_353_5,COL,South America,Colombia,11.433333,5974.501814,8.862860,3143,3,POINT (-71.8752334021 12.2387721729),3.0,4,4.0,5,3
4,BOX_116_353_6,COL,South America,Colombia,11.433333,5974.501814,8.862860,3143,3,POINT (-71.8748349513 12.2343999454),3.0,4,4.0,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,DOM,North America,Dominican Republic,27.800000,7939.336437,9.306864,1370,5,POINT (-71.6681862112 17.9652589019),5.0,3,3.0,4,5
37505,BOX_139_002_62,DOM,North America,Dominican Republic,27.800000,7939.336437,9.306864,1370,5,POINT (-71.6662284163 17.9614175026),5.0,3,3.0,4,5
37506,BOX_139_002_63,DOM,North America,Dominican Republic,27.800000,7939.336437,9.306864,1370,5,POINT (-71.664582619 17.9574419454),5.0,3,3.0,4,5
37507,BOX_139_002_64,DOM,North America,Dominican Republic,27.800000,7939.336437,9.306864,1370,5,POINT (-71.6629367897 17.953466310899998),5.0,3,3.0,4,5


In [27]:
Hazard_df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Future Hazards\Classified Data_2050_Low_emission.csv")
Hazard_df

,transect_id,country_id,continent,country_name,ESL_100yrRP_4.5_2050,Land_Subsidence_2040_high,Change_rate_SSP1_45_2050,geometry,ESL_100yrRP_4.5_2050 - Hazard level,Change_rate_SSP1_45_2050 - Hazard level
0,BOX_116_353_2,COL,South America,Colombia,1.196450,3.0,NaN,POINT (-71.8798045985 12.250683718),2.0,NaN
1,BOX_116_353_3,COL,South America,Colombia,1.196450,3.0,NaN,POINT (-71.87698143850001 12.247308305),2.0,NaN
2,BOX_116_353_4,COL,South America,Colombia,1.196450,3.0,NaN,POINT (-71.8756318462 12.2431443263),2.0,NaN
3,BOX_116_353_5,COL,South America,Colombia,1.196450,3.0,NaN,POINT (-71.8752334021 12.2387721729),2.0,NaN
4,BOX_116_353_6,COL,South America,Colombia,1.196450,3.0,NaN,POINT (-71.8748349513 12.2343999454),2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,DOM,North America,Dominican Republic,1.466509,2.0,0.255533,POINT (-71.6681862112 17.9652589019),4.0,1.0
37505,BOX_139_002_62,DOM,North America,Dominican Republic,1.466509,2.0,0.132781,POINT (-71.6662284163 17.9614175026),4.0,1.0
37506,BOX_139_002_63,DOM,North America,Dominican Republic,1.466509,2.0,0.059195,POINT (-71.664582619 17.9574419454),4.0,1.0
37507,BOX_139_002_64,DOM,North America,Dominican Republic,1.466509,2.0,-0.060692,POINT (-71.6629367897 17.953466310899998),4.0,1.0


Additive combination

In [28]:
scenario="_2050_Low_emission"
scenario_folder=r"\2050_Low_emission"


save_directory=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Combination\Risk_Level_FC"+scenario_folder
img_directory_path=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Combination\Risk_Level_FC\Plots"+scenario_folder

os.makedirs(save_directory, exist_ok=True)
os.makedirs(img_directory_path, exist_ok=True)

Discrete_Risk_Bins= True
Both_Plots=True

In [29]:
# UNCOMMENT AS NEEDED

gornitz_hazard_used=Hazard_df.columns[-1]  #Toggle as needed between -2 and -1




Value_to_show=title_name=gornitz_hazard_used.replace(" - Hazard level","")


title_name=gornitz_hazard_used.replace(" - Hazard level","")

gornitz_ESL=pd.concat([Hazard_df.iloc[:,0],Hazard_df[gornitz_hazard_used],Exposure_df.iloc[:,-5:],Vulnerability_df.iloc[:,-5:]],axis=1)
gornitz_ESL

,transect_id,Change_rate_SSP1_45_2050 - Hazard level,Percentage of Built Up Area_percentile - Exposure level_percentile,LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile,Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile,Population Within 1km of the coastline_percentile - Exposure level_percentile,Sediment Type_percentile - Exposure level_percentile,Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile,GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile,Mean Years of Schooling_Percentile - Vulnerability level_percentile,Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile,World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile
0,BOX_116_353_2,NaN,1,4,4.0,1,4.0,3.0,4,4.0,5,3
1,BOX_116_353_3,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3
2,BOX_116_353_4,NaN,1,5,4.0,1,4.0,3.0,4,4.0,5,3
3,BOX_116_353_5,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3
4,BOX_116_353_6,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5
37505,BOX_139_002_62,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5
37506,BOX_139_002_63,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5
37507,BOX_139_002_64,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5


In [30]:
# Define Hazard units to show on plot

if "ESL" in gornitz_hazard_used:
    units="m"
    PC_haz="ESL"
if "EWH" in gornitz_hazard_used:
    units="m"
    PC_haz="EWH"
if "Change_rate" in gornitz_hazard_used:
    units="m/yr"
    PC_haz="Change"
if "Land" in gornitz_hazard_used:
    units=""
    PC_haz="Land"
if "Cat3" in gornitz_hazard_used:
    units="years"
    PC_haz="Cat3"

Haz_Component= Hazard_df[Value_to_show].apply(lambda x: f"{Value_to_show} {x:.2f}{units}")
PC_haz

'Change'

In [31]:
indicator_num=len(gornitz_ESL.columns)-1
indicator_num

11

In [32]:
if indicator_num>9:
    var=" (Haz_Exp_Vul)"
else:
    var=" (Haz_Exp)"

In [33]:
gornitz_ESL_filled=gornitz_ESL.copy()
gornitz_ESL_filled["Risk Level"]=(gornitz_ESL[gornitz_hazard_used]+(np.nansum(gornitz_ESL.iloc[:,2:7],axis=1)/len(gornitz_ESL.iloc[0,2:7]))+(np.nansum(gornitz_ESL.iloc[:,-5:],axis=1)/len(gornitz_ESL.iloc[0,-5:])))/3
# gornitz_ESL_filled["Risk Level Class"] = np.round(gornitz_ESL_filled["Risk Level"])
gornitz_ESL_filled["Risk Level Class"] = np.nan
gornitz_ESL_filled
# 1.933493

,transect_id,Change_rate_SSP1_45_2050 - Hazard level,Percentage of Built Up Area_percentile - Exposure level_percentile,LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile,Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile,Population Within 1km of the coastline_percentile - Exposure level_percentile,Sediment Type_percentile - Exposure level_percentile,Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile,GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile,Mean Years of Schooling_Percentile - Vulnerability level_percentile,Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile,World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile,Risk Level,Risk Level Class
0,BOX_116_353_2,NaN,1,4,4.0,1,4.0,3.0,4,4.0,5,3,NaN,NaN
1,BOX_116_353_3,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN
2,BOX_116_353_4,NaN,1,5,4.0,1,4.0,3.0,4,4.0,5,3,NaN,NaN
3,BOX_116_353_5,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN
4,BOX_116_353_6,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5,2.600000,NaN
37505,BOX_139_002_62,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5,2.600000,NaN
37506,BOX_139_002_63,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5,2.666667,NaN
37507,BOX_139_002_64,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5,2.666667,NaN


In [34]:
#Classifying  Risk levels based in 20%, 40%, 60%, and 80& percentiles


RL1 = np.percentile(gornitz_ESL_filled["Risk Level"].dropna(), 20)
RL2 = np.percentile(gornitz_ESL_filled["Risk Level"].dropna(), 40)
RL3 = np.percentile(gornitz_ESL_filled["Risk Level"].dropna(), 60)
RL4 = np.percentile(gornitz_ESL_filled["Risk Level"].dropna(), 80)


gornitz_ESL_filled.loc[gornitz_ESL_filled["Risk Level"] <= RL1, "Risk Level Class"] = 1
gornitz_ESL_filled.loc[(gornitz_ESL_filled["Risk Level"] > RL1) & (gornitz_ESL_filled["Risk Level"] <= RL2), "Risk Level Class"] = 2
gornitz_ESL_filled.loc[(gornitz_ESL_filled["Risk Level"] > RL2) & (gornitz_ESL_filled["Risk Level"] <= RL3), "Risk Level Class"] = 3
gornitz_ESL_filled.loc[(gornitz_ESL_filled["Risk Level"] > RL3) & (gornitz_ESL_filled["Risk Level"] <= RL4), "Risk Level Class"] = 4
gornitz_ESL_filled.loc[gornitz_ESL_filled["Risk Level"] > RL4, "Risk Level Class"] = 5

gornitz_ESL_filled

,transect_id,Change_rate_SSP1_45_2050 - Hazard level,Percentage of Built Up Area_percentile - Exposure level_percentile,LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile,Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile,Population Within 1km of the coastline_percentile - Exposure level_percentile,Sediment Type_percentile - Exposure level_percentile,Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile,GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile,Mean Years of Schooling_Percentile - Vulnerability level_percentile,Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile,World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile,Risk Level,Risk Level Class
0,BOX_116_353_2,NaN,1,4,4.0,1,4.0,3.0,4,4.0,5,3,NaN,NaN
1,BOX_116_353_3,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN
2,BOX_116_353_4,NaN,1,5,4.0,1,4.0,3.0,4,4.0,5,3,NaN,NaN
3,BOX_116_353_5,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN
4,BOX_116_353_6,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5,2.600000,4.0
37505,BOX_139_002_62,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5,2.600000,4.0
37506,BOX_139_002_63,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5,2.666667,4.0
37507,BOX_139_002_64,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5,2.666667,4.0


In [35]:


df = gornitz_ESL.iloc[:,1:]

# Identify columns with NaN values
nan_columns = df.columns[df.isna().any()].tolist()

# Create a new column 'NaN_Columns' containing list of column names with NaN values for each row
gornitz_ESL_filled['Uncertainty_Columns'] = df.apply(lambda row: [col for col in nan_columns if pd.isna(row[col])], axis=1)
gornitz_ESL_filled['Uncertainty_Columns_length'] = gornitz_ESL_filled['Uncertainty_Columns'].apply(len)

df


,Change_rate_SSP1_45_2050 - Hazard level,Percentage of Built Up Area_percentile - Exposure level_percentile,LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile,Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile,Population Within 1km of the coastline_percentile - Exposure level_percentile,Sediment Type_percentile - Exposure level_percentile,Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile,GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile,Mean Years of Schooling_Percentile - Vulnerability level_percentile,Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile,World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile
0,NaN,1,4,4.0,1,4.0,3.0,4,4.0,5,3
1,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3
2,NaN,1,5,4.0,1,4.0,3.0,4,4.0,5,3
3,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3
4,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3
...,...,...,...,...,...,...,...,...,...,...,...
37504,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5
37505,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5
37506,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5
37507,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5


In [36]:

gornitz_ESL_filled["Confindence %"]=100*(indicator_num-gornitz_ESL_filled['Uncertainty_Columns'].apply(len))/indicator_num
gornitz_ESL_filled

,transect_id,Change_rate_SSP1_45_2050 - Hazard level,Percentage of Built Up Area_percentile - Exposure level_percentile,LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile,Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile,Population Within 1km of the coastline_percentile - Exposure level_percentile,Sediment Type_percentile - Exposure level_percentile,Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile,GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile,Mean Years of Schooling_Percentile - Vulnerability level_percentile,Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile,World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile,Risk Level,Risk Level Class,Uncertainty_Columns,Uncertainty_Columns_length,Confindence %
0,BOX_116_353_2,NaN,1,4,4.0,1,4.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091
1,BOX_116_353_3,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091
2,BOX_116_353_4,NaN,1,5,4.0,1,4.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091
3,BOX_116_353_5,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091
4,BOX_116_353_6,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5,2.600000,4.0,[],0,100.000000
37505,BOX_139_002_62,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5,2.600000,4.0,[],0,100.000000
37506,BOX_139_002_63,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5,2.666667,4.0,[],0,100.000000
37507,BOX_139_002_64,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5,2.666667,4.0,[],0,100.000000


In [37]:
# Read the CSV file into a DataFrame
circle_center_df = Exposure_df


# Identify the correct geometry column (replace 'geometry_column_name' with the actual column name)
geometry_column_name = 'geometry'
circle_center_df['x'] = circle_center_df[geometry_column_name].apply(lambda point_str: float(point_str.strip('POINT ()').split()[0]))
circle_center_df['y'] = circle_center_df[geometry_column_name].apply(lambda point_str: float(point_str.strip('POINT ()').split()[1]))

circle_center_df = circle_center_df.drop(columns=[geometry_column_name])

# Create the GeoDataFrame with the correct geometry column
gornitz_ESL_filled_gdf = gpd.GeoDataFrame(gornitz_ESL_filled, geometry=gpd.points_from_xy(circle_center_df.x, circle_center_df.y))



gornitz_ESL_filled_gdf

,transect_id,Change_rate_SSP1_45_2050 - Hazard level,Percentage of Built Up Area_percentile - Exposure level_percentile,LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile,Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile,Population Within 1km of the coastline_percentile - Exposure level_percentile,Sediment Type_percentile - Exposure level_percentile,Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile,GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile,Mean Years of Schooling_Percentile - Vulnerability level_percentile,Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile,World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile,Risk Level,Risk Level Class,Uncertainty_Columns,Uncertainty_Columns_length,Confindence %,geometry
0,BOX_116_353_2,NaN,1,4,4.0,1,4.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091,POINT (-71.87980 12.25068)
1,BOX_116_353_3,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091,POINT (-71.87698 12.24731)
2,BOX_116_353_4,NaN,1,5,4.0,1,4.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091,POINT (-71.87563 12.24314)
3,BOX_116_353_5,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091,POINT (-71.87523 12.23877)
4,BOX_116_353_6,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091,POINT (-71.87483 12.23440)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5,2.600000,4.0,[],0,100.000000,POINT (-71.66819 17.96526)
37505,BOX_139_002_62,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5,2.600000,4.0,[],0,100.000000,POINT (-71.66623 17.96142)
37506,BOX_139_002_63,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5,2.666667,4.0,[],0,100.000000,POINT (-71.66458 17.95744)
37507,BOX_139_002_64,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5,2.666667,4.0,[],0,100.000000,POINT (-71.66294 17.95347)


In [38]:
# import os
# from dotenv import load_dotenv

# # Load the .env file
# load_dotenv(r'C:\Users\rowe\Documents\GitHub\coclicodata\.env')

# # Access the Mapbox Access Token
# mapbox_access_token = os.getenv('MAPBOX_ACCESS_TOKEN')

# print("Mapbox Access Token:", mapbox_access_token)

In [39]:
img_lat = 18.1
img_lon = -77.3
view_width = 10

In [40]:
if Discrete_Risk_Bins:

    long_name="Risk Level Class"
    # color_scale = ['#33f510','#3CB371','#FFFF00', '#FFA500','#FF4500']
    label_dict = {
    1: "Very Low",
    2: "Low",
    3: "Medium",
    4: "High",
    5: "Very High"}

    colors = {'Very Low': '#33f510', 'Low': '#3CB371', 'Medium': '#FFFF00', 'High': '#FFA500', 'Very High': '#FF4500'}

    gornitz_ESL_filled_gdf[Value_to_show] = Haz_Component

    #Sorted for Risk level view
    sorted_CCN=gornitz_ESL_filled_gdf.sort_values(long_name)


    # Create scatter map plot
    fig_region = px.scatter_mapbox(sorted_CCN,
                            lat=sorted_CCN["geometry"].y,
                            lon=sorted_CCN["geometry"].x,
                            # hover_name=sorted_CCN[long_name].replace(label_dict),
                            hover_name=Value_to_show,
                            color=sorted_CCN[long_name].replace(label_dict),
                            # color=sorted_CCN[long_name],
                            color_discrete_map=colors,
                            mapbox_style="carto-positron",width=2000,
                            height=1200)


    # Update map layout to adjust center and zoom
    fig_region.update_layout(mapbox=dict(center=dict(lat=img_lat + 1, lon=img_lon + 4), zoom=5.5),
                    title=f"{gornitz_ESL_filled_gdf.columns[1].replace('Hazard', 'Risk')}{var} Discrete Distribution along Caribbean's shoreline")

    fig_region .update_layout(legend=dict(
    traceorder='normal',
    title=long_name,
    title_font=dict(size=16),
    itemsizing='constant',))
    fig_region.show()


    fig_name_shoreline=gornitz_ESL_filled_gdf.columns[1].replace('Hazard', 'Risk') +var+"Discrete- Caribbean.png"
    pio.write_image(fig_region, os.path.join(img_directory_path, fig_name_shoreline))

### IMPORT MAX AND MIN TICK RANGES FROM THE PRESENT CLIMATE ANALYSIS (FOR PLOTTING ONLY)

In [41]:
PC_Risk_path=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Combination\Risk_Level_PC"

files = os.listdir(PC_Risk_path)

# Filter only CSV files
csv_files = [file for file in files if file.endswith('.csv')]

csv_files
pc_haz_file = [colu for colu in csv_files if PC_haz in colu ]
pc_haz_file=pc_haz_file[0]
print(pc_haz_file)

PC_Risk=pd.read_csv(os.path.join(PC_Risk_path,pc_haz_file))
PC_Risk.head(2)


Shoreline_Change_Rate_PC_percentile - Risk level.csv


,transect_id,Shoreline_Change_Rate_PC_percentile - Hazard level,Percentage of Built Up Area_percentile - Exposure level_percentile,LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile,Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile,Population Within 1km of the coastline_percentile - Exposure level_percentile,Sediment Type_percentile - Exposure level_percentile,Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile,GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile,Mean Years of Schooling_Percentile - Vulnerability level_percentile,Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile,World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile,Risk Level,Risk Level Class,Uncertainty_Columns,Uncertainty_Columns_length,Confindence %,geometry,Shoreline_Change_Rate_PC_percentile
0,BOX_116_353_2,5.0,3,4,5.0,2,4.0,3.0,4,4.0,5,3,4.133333,5.0,[],0,100.0,POINT (-71.8798045985 12.250683718),Shoreline_Change_Rate_PC_percentile -16.44years
1,BOX_116_353_3,1.0,1,5,5.0,1,2.0,3.0,4,4.0,5,3,2.533333,4.0,[],0,100.0,POINT (-71.87698143850001 12.247308305),Shoreline_Change_Rate_PC_percentile 0.34years


In [42]:

if not Discrete_Risk_Bins or Both_Plots:

    long_name = "Risk Level"

    print_val=gornitz_ESL_filled_gdf.columns[1].replace('Hazard', 'Risk') 
    # Define tick names

    lat = 18.1
    lon = -77.3
    view_width = 10
    # Define tick names
    tick_names = ["Very Low", "Low", "Moderate", "High", "Very High"]

    tk1=np.min(PC_Risk[long_name].dropna())
    tk5=np.max(PC_Risk[long_name].dropna())
    tk3=np.mean([tk1,tk5])
    tk4=np.mean([tk3,tk5])
    tk2=np.mean([tk3,tk1])


    
    
    tick_vals= [tk1, tk2,tk3, tk4, tk5]
   

    # Define parameters

    color_scale = ['#33f510', '#3CB371', '#FFFF00', '#FFA500', '#FF4500']

    # Update Hazard Component values
    gornitz_ESL_filled_gdf[Value_to_show] = Haz_Component

    # Sort values for Risk level view
    sorted_CCN = gornitz_ESL_filled_gdf.sort_values(long_name)

    #drop nans for shoreline change
    sorted_CCN=sorted_CCN[~sorted_CCN[long_name].isna()].copy()

    # Create scatter map plot
    fig_region = px.scatter_mapbox(sorted_CCN,
                                    lat=sorted_CCN["geometry"].y,
                                    lon=sorted_CCN["geometry"].x,
                                    hover_name=Value_to_show,
                                    color=sorted_CCN[long_name],
                                    color_continuous_scale=color_scale,
                                    range_color=(tk1, tk5),
                                    mapbox_style="carto-positron",
                                    width=2000,
                                    height=1200)

    # Update the tick labels
    fig_region.update_layout(coloraxis_colorbar=dict(title=long_name, tickvals=tick_vals, ticktext=tick_names, tickfont=dict(size=15, color='black')))

    fig_region.update_layout(    mapbox=dict(
            zoom=5,
            center=dict(lat=17.9, lon=-71.5)
        ),	title=f"{print_val} Distribution along Caribbean's shoreline")
    fig_region.update_traces(marker=dict(size=7))
    fig_region.update_layout(legend=dict(
        traceorder='normal',
        # title=long_name,
        title_font=dict(size=14),
        itemsizing='constant',
    
    ))
    fig_region.update_layout(
        mapbox_style="carto-positron",
        mapbox_layers=[
            {
                "below": 'traces',
                "sourcetype": "raster",
                "sourceattribution": "ESRI World Imagery",
                "source": [
                    "https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
                ]
            },
            # {
            #     "below": 'traces',
            #     "sourcetype": "raster",
            #     "sourceattribution": "Google Maps",
            #     "source": [
            #         "http://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}"
            #     ]
            # },
            # {
            #     "below": 'traces',
            #     "sourcetype": "raster",
            #     "sourceattribution": "United States Geological Survey",
            #     "source": [
            #         "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            #     ]
            # },
            # {
            #     "below": 'traces',
            #     "sourcetype": "raster",
            #     "sourceattribution": "Bing Maps Satellite Imagery",
            #     "source": [
            #         "https://t0.tiles.virtualearth.net/tiles/a{q}.jpeg?g=685&mkt=en-us&n=z"
            #     ]
            # },
        

        ]
        )

    fig_region.update_layout(
        mapbox=dict(center=dict(lat=lat + 1, lon=lon + 4), zoom=5.3),
        title=dict(
            # text="Infant Mortality Rate along Caribbean's shoreline",
            font=dict(size=25),
            x=0.5,  # Centers the title
            xanchor='center'  # Aligns the title in the center
        )
    )


    # Show the figure
    fig_region.show()


    # Save the figure
    fig_name_shoreline = gornitz_ESL_filled_gdf.columns[1].replace('Hazard', 'Risk') + var + "Continuous- Caribbean.png"
    pio.write_image(fig_region, os.path.join(img_directory_path, fig_name_shoreline))


In [43]:
gornitz_ESL_filled_gdf

,transect_id,Change_rate_SSP1_45_2050 - Hazard level,Percentage of Built Up Area_percentile - Exposure level_percentile,LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile,Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile,Population Within 1km of the coastline_percentile - Exposure level_percentile,Sediment Type_percentile - Exposure level_percentile,Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile,GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile,Mean Years of Schooling_Percentile - Vulnerability level_percentile,Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile,World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile,Risk Level,Risk Level Class,Uncertainty_Columns,Uncertainty_Columns_length,Confindence %,geometry,Change_rate_SSP1_45_2050
0,BOX_116_353_2,NaN,1,4,4.0,1,4.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091,POINT (-71.87980 12.25068),Change_rate_SSP1_45_2050 nanm/yr
1,BOX_116_353_3,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091,POINT (-71.87698 12.24731),Change_rate_SSP1_45_2050 nanm/yr
2,BOX_116_353_4,NaN,1,5,4.0,1,4.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091,POINT (-71.87563 12.24314),Change_rate_SSP1_45_2050 nanm/yr
3,BOX_116_353_5,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091,POINT (-71.87523 12.23877),Change_rate_SSP1_45_2050 nanm/yr
4,BOX_116_353_6,NaN,1,5,4.0,1,2.0,3.0,4,4.0,5,3,NaN,NaN,[Change_rate_SSP1_45_2050 - Hazard level],1,90.909091,POINT (-71.87483 12.23440),Change_rate_SSP1_45_2050 nanm/yr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5,2.600000,4.0,[],0,100.000000,POINT (-71.66819 17.96526),Change_rate_SSP1_45_2050 0.26m/yr
37505,BOX_139_002_62,1.0,1,4,3.0,2,4.0,5.0,3,3.0,4,5,2.600000,4.0,[],0,100.000000,POINT (-71.66623 17.96142),Change_rate_SSP1_45_2050 0.13m/yr
37506,BOX_139_002_63,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5,2.666667,4.0,[],0,100.000000,POINT (-71.66458 17.95744),Change_rate_SSP1_45_2050 0.06m/yr
37507,BOX_139_002_64,1.0,1,5,3.0,2,4.0,5.0,3,3.0,4,5,2.666667,4.0,[],0,100.000000,POINT (-71.66294 17.95347),Change_rate_SSP1_45_2050 -0.06m/yr


In [44]:
gornitz_ESL_filled_gdf.to_csv(save_directory+f"/{gornitz_ESL_filled_gdf.columns[1].replace('Hazard', 'Risk')}.csv",index=False)

Curacao Multi-Criteria Risk Level for Extreme Sea Level Hazard Using Mclaughlin and Cooper (2010)